# 第14回 仕様とテスト

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-ueno/begin-python-2025/blob/master/workbook/lecture14_new.ipynb)

___

## この講義で学ぶこと

これまでの講義では、プログラムが正しく動く「正常系」の書き方を中心に学んできた。
しかし実際のプログラムでは、予期しないエラーが発生することがある。
今回は、プログラムの**仕様**を明確にし、異常事態に対応する**例外処理**を学び、プログラムが仕様通り動くことを確認する**テスト**の書き方を学ぶ。

これらは、どのプログラミング言語でも使える汎用的な知識である。
また、次回の講義で学ぶ生成AIとの付き合い方においても、仕様とテストの考え方は非常に重要になる。

## 仕様とは何か

プログラムの**仕様**とは、そのプログラムが「何をするものか」を明確に定めたものである。
これまで作成してきた関数にも、実は「仕様」がある。

例えば、以下のような割り算をする関数を考えてみよう。

In [ ]:
def divide(a, b):
    return a / b

この関数の仕様は何だろうか。以下の点を考えてみよう：

- 何を受け取るのか（引数）
- 何を返すのか（戻り値）
- どんな場合に失敗するのか（制約）

実際に使ってみよう。

In [ ]:
print(divide(10, 2))  # 正常系: 5.0が期待される

In [ ]:
print(divide(10, 0))  # 異常系: エラーが発生する

`ZeroDivisionError`というエラーが発生した。このエラーを防ぐためには、仕様を明確にする必要がある。

仕様を明文化すると、以下のようになる：

```
関数名: divide
引数: a (数値), b (数値)
戻り値: a を b で割った結果（数値）
前提条件: b は 0 でないこと
```

Pythonでは、関数の仕様を**ドキュメンテーション文字列**（docstring）として記述するのが一般的である。

In [ ]:
def divide(a, b):
    """
    a を b で割った値を返す。

    引数:
        a: 被除数（数値）
        b: 除数（数値、0以外）

    戻り値:
        a / b の計算結果（数値）

    前提条件:
        b は 0 でないこと
    """
    return a / b

ドキュメンテーション文字列を書いておくと、`help()` 関数で確認できる。

In [ ]:
help(divide)

## 例外処理

### 例外とは

プログラム実行中に発生するエラーのことを**例外**（exception）という。
先ほどの `ZeroDivisionError` も例外の一種である。

例外が発生すると、通常はプログラムが停止してしまう。
しかし、**例外処理**を行うことで、エラーが発生しても適切に対処し、プログラムを続行させることができる。

例外処理は、if文のような制御構造の一種として理解できる。

### try-except構文

例外処理を行うには、`try-except` 構文を使う。基本的な形は以下の通りである：

```python
try:
    # エラーが発生する可能性のあるコード
except エラーの種類:
    # エラーが発生したときの処理
```

`divide` 関数を例外処理を使って安全にしてみよう。

In [ ]:
def safe_divide(a, b):
    """
    a を b で割った値を返す。b が 0 の場合は None を返す。

    引数:
        a: 被除数（数値）
        b: 除数（数値）

    戻り値:
        a / b の計算結果（数値）、または None
    """
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print("エラー: 0で割ることはできません")
        return None

In [ ]:
print(safe_divide(10, 2))   # 5.0
print(safe_divide(10, 0))   # エラーメッセージを表示して None を返す

### 複数の例外処理

複数の種類の例外を処理したい場合は、`except` を複数書くことができる。

例えば、ユーザーからの入力を整数に変換する関数を考えてみよう。

In [ ]:
def safe_int_convert(text):
    """
    文字列を整数に変換する。変換できない場合は None を返す。

    引数:
        text: 変換する文字列

    戻り値:
        整数、または None
    """
    try:
        value = int(text)
        return value
    except ValueError:
        print(f"エラー: '{text}' は整数に変換できません")
        return None

In [ ]:
print(safe_int_convert("123"))    # 123
print(safe_int_convert("abc"))    # エラーメッセージを表示して None
print(safe_int_convert("12.5"))   # エラーメッセージを表示して None

**練習1**  
ファイルを読み込む以下の関数に、例外処理を追加しなさい。ファイルが見つからない場合は `None` を返すようにすること。

ヒント: ファイルが見つからないときは `FileNotFoundError` が発生する。

In [ ]:
def read_file(filename):
    """
    ファイルの内容を読み込んで返す。
    ファイルが見つからない場合は None を返す。
    """
    # ここに適切なコードを書く
    pass

In [ ]:
# テスト（存在しないファイル名で試す）
result = read_file("存在しないファイル.txt")
print(result)  # None が表示されれば成功

```{toggle}
**解答例**   
<pre style={"white-space": "pre"}>
def read_file(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
            return content
    except FileNotFoundError:
        print(f"エラー: {filename} が見つかりません")
        return None
</pre>
```

## テストとは

### テストの必要性

さて、プログラムが仕様通りに動くことを確認するにはどうすればよいだろうか？

ナイーブな方法に、人間がプログラムを実行し結果を見る方法がある。しかし人間は間違いもあるし毎回結果を確認するのは大変である。

そこで、ソフトウェア開発においては、**テスト**を行う。

テストとは、プログラムの動作を確認するための別のプログラムのことである。



プログラムが仕様通りに動くことを確認するために、**テスト**を行う。
テストとは、プログラムの動作を確認するための別のプログラムのことである。

プログラムが仕様通りに動くことを確認するための別のプログラムを、**テスト** と呼ぶ。

テストを書くことには、以下のようなメリットがある：

1. **仕様が明確になる**：どう動くべきかが明確になる
1. **変更に強くなる**：後で変更しても、テストで確認できる
1. **バグを早期に発見できる**：問題を早く見つけられる
1. **ドキュメントになる**：使い方の例が分かる

### assert文を使ったテスト

最もシンプルなテストの方法は、`assert`文を使うことである。
`assert` は「〜であることを確認する」という意味で、条件が `False` のときにエラーを発生させる。

基本的な形は以下の通り：

```python
assert 条件式
```

簡単な例で試してみよう。

In [ ]:
# これは成功する（何も起きない）
assert 2 + 2 == 4
print("テスト成功")

In [ ]:
# これは失敗する（AssertionError が発生）
assert 2 + 2 == 5
print("この行は実行されない")

### 関数のテスト

関数をテストする場合は、期待される結果と実際の結果を比較する。
簡単な足し算の関数でテストを書いてみよう。

In [ ]:
def add(a, b):
    """2つの数を足す"""
    return a + b

# 正常系のテスト
assert add(2, 3) == 5
assert add(0, 0) == 0
assert add(-1, 1) == 0
assert add(100, 200) == 300

print("全てのテストが成功しました！")

### 正常系と異常系のテスト

テストには、**正常系**（期待通りの入力）と**異常系**（予期しない入力）の両方を含めるべきである。

先ほどの `safe_divide` 関数のテストを書いてみよう。

In [ ]:
# 正常系のテスト
assert safe_divide(10, 2) == 5
assert safe_divide(7, 2) == 3.5
assert safe_divide(0, 5) == 0

# 異常系のテスト
assert safe_divide(10, 0) is None
assert safe_divide(0, 0) is None

print("全てのテストが成功しました！")

**練習2**  
以下の仕様を満たす `get_first` 関数を実装し、その後テストを書いて確認しなさい。

```
仕様:
  関数名: get_first
  引数: lst (リスト)
  戻り値: リストの最初の要素、リストが空の場合は None
```

In [ ]:
def get_first(lst):
    """リストの最初の要素を返す。空の場合は None を返す。"""
    # ここに実装を書く
    pass

In [ ]:
# ここにテストを書く（正常系3つ、異常系1つ以上）
# 例: assert get_first([1, 2, 3]) == 1


```{toggle}
**解答例**   
<pre style={"white-space": "pre"}>
def get_first(lst):
    if len(lst) == 0:
        return None
    return lst[0]

# テスト
assert get_first([1, 2, 3]) == 1
assert get_first(["a", "b"]) == "a"
assert get_first([100]) == 100
assert get_first([]) is None
print("全てのテストが成功しました！")
</pre>
```

## テストと仕様の関係

### 従来の開発フロー

これまでは、以下のような流れでプログラムを作成してきた：

1. 仕様を考える
2. コードを書く
3. テストする

しかし、実際のソフトウェア開発では、**仕様を最初から完璧に決めるのは非常に困難**である。

### テストファーストの考え方

近年では、以下のような開発スタイルが注目されている：

1. **テストを先に書く**
2. テストが通るようにコードを書く
3. コードを改善する

この方法は**テスト駆動開発**（Test-Driven Development, TDD）と呼ばれる。

テストを先に書くことで、以下のメリットがある：

- **仕様が明確になる**：何を作るべきかがはっきりする
- **必要最小限のコード**：テストを満たす最小のコードを書ける
- **バグが少ない**：最初からテストがあるので安心

### 例：文字列処理関数をTDDで作る

文字列から空白を全て削除する関数を、テストファーストで作ってみよう。

**Step 1: まずテストを書く**（この時点では関数は存在しない）

In [ ]:
# テストを先に書く
def test_remove_spaces():
    assert remove_spaces("hello world") == "helloworld"
    assert remove_spaces("  a  b  ") == "ab"
    assert remove_spaces("") == ""
    assert remove_spaces("abc") == "abc"
    print("全てのテストが成功しました！")

# この時点ではまだ関数が存在しないので、実行するとエラーになる
# test_remove_spaces()  # NameError: name 'remove_spaces' is not defined

**Step 2: テストを満たすようにコードを書く**

In [ ]:
def remove_spaces(text):
    """文字列から空白を全て削除する"""
    return text.replace(" ", "")

**Step 3: テストを実行**

In [ ]:
test_remove_spaces()  # 成功！

**練習3**  
回文（前から読んでも後ろから読んでも同じ文字列）かどうかを判定する関数を、テストファーストで作成しなさい。

まず以下のテストが通るように `is_palindrome` 関数を実装すること。

ヒント: 文字列を反転するには `text[::-1]` を使う。空白を無視し、大文字小文字を区別しない場合は、`text.replace(" ", "").lower()` で前処理する。

In [ ]:
# Step 1: まずテストを書く
def test_is_palindrome():
    assert is_palindrome("racecar") == True
    assert is_palindrome("hello") == False
    assert is_palindrome("") == True
    assert is_palindrome("a") == True
    print("全てのテストが成功しました！")

In [ ]:
# Step 2: テストが通るように is_palindrome 関数を実装しなさい
def is_palindrome(text):
    # ここに実装を書く
    pass

In [ ]:
# Step 3: テストを実行
test_is_palindrome()

```{toggle}
**解答例**   
<pre style={"white-space": "pre"}>
def is_palindrome(text):
    return text == text[::-1]
</pre>
```

## 生成AIとの関係

### 仕様が曖昧だとAIも困る

次回の講義では、生成AI（ChatGPTやClaude等）との付き合い方を学ぶ。
生成AIにコードを書いてもらう際、**仕様が曖昧だと、AIも何を作ればいいか分からない**。

例えば、以下のような曖昧な指示を考えてみよう：

> 「数を足す関数を作って」

AIは以下のどれを作るべきか判断できない：

```python
# 2つの数を足す？
def add(a, b):
    return a + b

# 複数の数を足す？
def add(*args):
    return sum(args)

# リストの要素を全て足す？
def add(numbers):
    return sum(numbers)
```

仕様を明確にすることで、AIに正確な指示を出せる。

### テストがあればAIの出力を検証できる

テストを書いてAIに渡すことで、AIが生成したコードが正しいかを自動的に検証できる。

例えば、以下のようにテストと一緒に指示を出す：

> 「以下のテストが通る関数を作ってください：
> 
> ```python
> assert count_vowels("hello") == 2
> assert count_vowels("AEIOU") == 5
> assert count_vowels("xyz") == 0
> ```
> 」

このようにすれば、AIが何を作るべきかが明確になり、生成されたコードが正しいかもテストで確認できる。

### AI時代における人間の役割

生成AIが発達しても、人間には以下の役割がある：

1. **仕様を考える**：何を作りたいのかを明確にする
2. **テストを考える**：期待する動作を定義する
3. **検証する**：AIの出力が正しいかを確認する
4. **問題を分解する**：複雑な問題を小さな部分に分ける

これらはどれも、プログラミングの基礎的な知識がないとできないことである。
次回の講義では、これらの点をさらに詳しく学ぶ。

参考：日経

## 演習

**課題1**  
以下の仕様を満たす `calculate_grade` 関数を実装し、正常系と異常系の両方のテストを書きなさい。

```
仕様:
  関数名: calculate_grade
  引数: score (整数)
  戻り値: 評価（文字列）
    - 90以上: "A"
    - 80以上90未満: "B"  
    - 70以上80未満: "C"
    - 60以上70未満: "D"
    - 60未満: "F"
    - 0未満または100超: None（異常値）
```

In [ ]:
def calculate_grade(score):
    """
    点数から成績評価を返す。
    0未満または100超の場合は None を返す。
    """
    # ここに実装を書く
    pass

In [ ]:
# ここにテストを書く（正常系5つ、異常系2つ以上）
# 例: assert calculate_grade(95) == "A"


**課題2**  
以下のテストが通るように `find_max` 関数を実装しなさい。

In [ ]:
# まずテストを確認する
def test_find_max():
    assert find_max([1, 5, 3, 2]) == 5
    assert find_max([10]) == 10
    assert find_max([-1, -5, -3]) == -1
    assert find_max([100, 200, 150]) == 200
    assert find_max([]) is None
    print("全てのテストが成功しました！")

In [ ]:
# テストが通るように find_max 関数を実装しなさい
def find_max(numbers):
    """
    リストの中の最大値を返す。
    リストが空の場合は None を返す。
    """
    # ここに実装を書く
    pass

In [ ]:
# テストを実行
test_find_max()

**課題3（発展）**  
辞書のリストから、特定のキーの値のみを抽出する関数をテストファーストで作成しなさい。

例：
```python
data = [{"name": "Alice", "age": 20}, {"name": "Bob", "age": 25}]
extract_values(data, "name")  # ["Alice", "Bob"] が返る
```

Step 1: まずテストを書く  
Step 2: テストが通るように関数を実装する  
Step 3: テストを実行して確認する

In [ ]:
# Step 1: テストを書く
def test_extract_values():
    # ここにテストを書く
    pass

In [ ]:
# Step 2: 関数を実装する
def extract_values(data, key):
    # ここに実装を書く
    pass

In [ ]:
# Step 3: テストを実行
test_extract_values()